## Adding Information to a Database
### Getting and Saving Data
*Curtis Miller*

In this notebook I create and add data to a database table.

First, load in required libraries.

In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from pandas import DataFrame

In [ ]:
def pymysql_sqlalchemy_stringgen(user, passwd, host, dbname):
    """Generate a connection string for use with SQLAlchemy for MySQL and PyMySQL connections
    
    Args:
        user (str): The username of the connecting user
        passwd (str): The user's password
        host (str): The host for where the database is located
        dbname (str): The name of the database to connect with
    
    Returns:
        (str) A SQLAlchemy connection string suitable for use with create_engine()
    
    Additional options for the connection are not supported with this function.
    """
    
    return "mysql+pymysql://" + user + ":" + passwd + "@" + host + "/" + dbname

In [ ]:
conn = create_engine(pymysql_sqlalchemy_stringgen("root", pswd, "localhost", "poppyramids")).connect()    # Connect to database

Read in a pandas `DataFrame` with the information we want to add to the database.

In [ ]:
pops = pd.read_csv("PopPyramids.csv", index_col=["Country", "Year", "Age"])
pops.head()

These column names could be trouble in a database, so let's make names lower case and use `_` instead of spaces.

In [ ]:
pops.columns = pd.Index(pd.Series(pops.columns).map(lambda x: x.lower().replace(" ", "_")))    # Format column names
pops.index.rename([n.lower() for n in pops.index.names], inplace=True)    # Rename the names of the MultiIndex levels

In [ ]:
pops.head()

Create a table in the database using SQL to hold this data; it starts with no rows.

In [ ]:
make_table = """CREATE TABLE `populations` (
                    `region` varchar(20),
                    `year` int(4),
                    `both_sexes_population` bigint(20),
                    `male_population` bigint(20),
                    `female_population` bigint(20),
                    `percent_both_sexes` double,
                    `percent_male` double,
                    `percent_female` double,
                    `sex_ratio` double,
                    `age` char(5) NOT NULL,
                    `country` char(28) NOT NULL,
                    PRIMARY KEY (`country`,`year`,`age`)
                );"""

conn.execute(make_table)

Add the rows to the table, appending them. Keys will be properly matched.

In [ ]:
pops.to_sql("populations",         # The name of the table in the database
            con=conn,              # The connection object
            if_exists='append')    # If the table exists, add rows (other options: 'fail' to do nothing, 'replace' to 
                                   # delete and make new table)

Under the hood, `to_sql()` is issuing `INSERT` commands to MySQL.

In [ ]:
conn.close()    # Always close the connection